In [7]:
import numpy as np
import string
from collections import defaultdict, Counter

# Initialize dictionaries with defaultdict for better efficiency
pi = Counter()
A1 = defaultdict(list)
A2 = defaultdict(list)
A3 = defaultdict(list)

# Function to process transitions more efficiently
def fill_dict(d, k, v):
    d[k].append(v)

# Read the file efficiently
with open('YiLei_Poem.txt', 'r', encoding='utf-8') as f:
    for line in f:
        tokens = line.rstrip().lower().split()
        T = len(tokens)

        if T > 2:
            for i in range(T):
                t = tokens[i]

                if i == 0:
                    pi[t] += 1
                elif i == 1:
                    fill_dict(A1, tokens[i - 1], t)
                elif i == 2:
                    fill_dict(A2, (tokens[i - 2], tokens[i - 1]), t)
                elif i == T - 1:
                    fill_dict(A3, (tokens[i - 3], tokens[i - 2], tokens[i - 1]), '<end>')
                else:
                    fill_dict(A3, (tokens[i - 3], tokens[i - 2], tokens[i - 1]), t)

# Normalize the initial distribution
pi_total = sum(pi.values())
pi = {t: count / pi_total for t, count in pi.items()}

# Convert lists to probability dictionaries
def list_to_probdict(tokens):
    counts = Counter(tokens)
    total = sum(counts.values())
    return {t: c / total for t, c in counts.items()}

A1 = {t_1: list_to_probdict(token) for t_1, token in A1.items()}
A2 = {k: list_to_probdict(v) for k, v in A2.items()}
A3 = {k: list_to_probdict(v) for k, v in A3.items()}

# Function to sample a word based on probabilities
def sample_word(d):
    p0 = np.random.random()
    cumulative = 0
    for t, p in d.items():
        cumulative += p
        if p0 < cumulative:
            return t
    return None

# Function to generate a poem
def generate_Poem():
    for _ in range(7):  # Generate 7 lines
        sentence = []

        # Initial word
        w0 = sample_word(pi)
        sentence.append(w0)

        # Sample second and third words
        if w0 in A1:
            w1 = sample_word(A1[w0])
            sentence.append(w1)
        else:
            continue

        if (w0, w1) in A2:
            w2 = sample_word(A2[(w0, w1)])
            sentence.append(w2)
        else:
            continue

        # Sample remaining words
        while (w0, w1, w2) in A3:
            w3 = sample_word(A3[(w0, w1, w2)])
            if w3 == '<end>':
                break
            sentence.append(w3)
            w0, w1, w2 = w1, w2, w3  # Shift words forward

        print(' '.join(sentence))

generate_Poem()


who has seen through the awful
my heart with mud. an old
slip on your frivolous
from all directions
this bliss is yours,
by daylight, moonlight.
fistfuls of rain fall hard,
